In [ ]:
!pip3 install transformers

In [6]:
import pandas as pd


df_muslim = pd.read_csv("toxicbias_train.csv")
df_muslim = df_muslim[df_muslim['rationale'] == 'prejudice against muslims']
df_neutral = pd.read_csv("toxicbias_train.csv")
df_neutral = df_neutral[df_neutral['bias'] == 'neutral'].sample(df_muslim.shape[0])

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from src.Helpers import *
import torch

DEVICE = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")

import pandas as pd
import torch
import time


def standardise_results(results):
    label_mapping = {'contradiction': results['contradiction'],
                     'neutral': results['neutral'],
                     'entailment': results['entailment']}
    return label_mapping


def convert_probabilities(probabilities, label_mapping):
    probabilities_list = probabilities.tolist()[0]  # Convert the tensor to a list and extract the first (and only) batch
    return {name: round(float(pred) * 100, 1) for pred, name in zip(probabilities_list, label_mapping)}



def get_random_samples(csv_filename, num_samples):
    df = pd.read_csv(csv_filename)
    random_samples = df.sample(n=num_samples)
    return random_samples


def benchmark_test(premise, hypothesis, model, num_runs):
    # Setup device
    if torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")

    results = []
    for _ in range(num_runs):
        start_time = time.time()
        _, _ = model(premise, hypothesis)
        execution_time = time.time() - start_time
        results.append(execution_time)

    avg_execution_time = sum(results) / len(results)
    return device, avg_execution_time


In [34]:
premises_bias = df_muslim['comment_text'].values
premises_bias = [str(p) for p in premises_bias]

premises_neutral = df_neutral['comment_text'].values
premises_neutral = [str(p) for p in premises_neutral]

hypotheses = [
  'The use of violence is encouraged or desired against a religion.',
  'A religion should be discriminated against.',
  'Members of a religion should be excluded and marginalized.',
  'Members of a religion should remain in their respective countries.',
  'Members of a religion are compared to animals.',
  'A religion or members of a religion are compared to devils or demons.',
  'The integration of a religion should be avoided in a country or society.',
  'A religion or members of a religion are stupid.',
  'A religion is inferior when compared to another religion.',
  'Religious practices of a particular faith should be restricted or banned in public spaces.',
  'Religious attire or symbols should be prohibited.',
  'A religion promotes violence or intolerance.',
  'Members of a religion are a threat.',
  'Members of a religion are not trustworthy or loyal.'
 ]


In [9]:
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_colwidth', 500)  # Display full text in columns

In [45]:
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Assuming you have already defined DEVICE and the convert_probabilities and standardise_results functions.

def bart_nli_nested(premises, hypotheses):
    model_name = 'facebook/bart-large-mnli'
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    model.to(DEVICE) 

    # Convert scalar inputs to lists
    if not isinstance(premises, (list, tuple)):
        premises = [premises]
    if not isinstance(hypotheses, (list, tuple)):
        hypotheses = [hypotheses]

    num_premises = len(premises)
    num_hypotheses = len(hypotheses)
    results = {}

    for i in range(num_premises):
        labels = []
        probabilities_list = []

        for j in range(num_hypotheses):
            # Tokenize the input pair
            inputs = tokenizer(premises[i], hypotheses[j], return_tensors='pt', padding=True, truncation=True)

            # Run the input through the model
            logits = model(**inputs.to(DEVICE)).logits

            # Get probabilities and labels for the input
            probabilities = torch.softmax(logits, dim=1)
            label_mapping = ['contradiction', 'neutral', 'entailment']
            label = label_mapping[probabilities.argmax(dim=1)]

            probabilities = convert_probabilities(probabilities, label_mapping)

            labels.append(label)
            probabilities_list.append(standardise_results(probabilities))

        results[premises[i]] = (labels, probabilities_list)

    return results


In [46]:
df = df_muslim.copy()
results = bart_nli_nested(premises_bias, hypotheses)

# Process the results separately and add them to the dataframe
labels_column = []
probabilities_column = []

for premise in df['comment_text']:
    if premise in results:
        labels, probabilities = results[premise]
    else:
        labels, probabilities = None, None
    labels_column.append(labels)
    probabilities_column.append(probabilities)

df['labels'] = labels_column
df['probabilities'] = probabilities_column

filtered_df = df.dropna(subset=['labels'])



In [48]:
# Function to check if 'entailment' is present in a row
def has_entailment(row):
    return 'entailment' in row

# Apply the function to each row in the 'results' column
filtered_df['has_entailment'] = filtered_df['labels'].apply(has_entailment)

# Count the number of rows with 'entailment' and the ones without
num_rows_with_entailment = filtered_df['has_entailment'].sum()
num_rows_without_entailment = filtered_df.shape[0] - num_rows_with_entailment

print("Rows with 'entailment':", num_rows_with_entailment)
print("Rows without 'entailment':", num_rows_without_entailment)

Rows with 'entailment': 16
Rows without 'entailment': 4


/var/folders/yy/fwc73zl141q0zj_v307n1l9w0000gn/T/ipykernel_16477/3429989658.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['has_entailment'] = filtered_df['labels'].apply(has_entailment)


In [ ]:
df = df_neutral.copy()
results = bart_nli_nested(premises_neutral, hypotheses)

# Process the results separately and add them to the dataframe
labels_column = []
probabilities_column = []

for premise in df['comment_text']:
    if premise in results:
        labels, probabilities = results[premise]
    else:
        labels, probabilities = None, None
    labels_column.append(labels)
    probabilities_column.append(probabilities)

df['labels'] = labels_column
df['probabilities'] = probabilities_column

filtered_df_neutral = df.dropna(subset=['labels'])



In [ ]:
# Function to check if 'entailment' is present in a row
def has_entailment(row):
    return 'entailment' in row

# Apply the function to each row in the 'results' column
filtered_df_neutral['has_entailment'] = filtered_df_neutral['labels'].apply(has_entailment)

# Count the number of rows with 'entailment' and the ones without
num_rows_with_entailment = filtered_df_neutral['has_entailment'].sum()
num_rows_without_entailment = filtered_df_neutral.shape[0] - num_rows_with_entailment

print("Rows with 'entailment':", num_rows_with_entailment)
print("Rows without 'entailment':", num_rows_without_entailment)

In [ ]:
import pandas as pd
from sklearn.metrics import f1_score

bias_df = pd.DataFrame(filtered_df)
neutral_df = pd.DataFrame(filtered_df_neutral)

# Define the criterion for the biased DataFrame (all results should be 'entailment')
def is_correct_bias(row):
    return any(label == 'entailment' for label in row)

# Define the criterion for the neutral DataFrame (all results should be 'contradiction' or 'neutral')
def is_correct_neutral(row):
    return all(label != 'entailment' for label in row)

# Apply the functions to create binary arrays indicating correctness in both DataFrames
bias_df['is_correct'] = bias_df['labels'].apply(is_correct_bias)
neutral_df['is_correct'] = neutral_df['labels'].apply(is_correct_neutral)

# Combine both DataFrames
combined_df = pd.concat([bias_df, neutral_df], ignore_index=True)
combined_df['predicted'] = True

# Calculate the F1 score for the combined DataFrame
f1 = f1_score(combined_df['is_correct'], combined_df['predicted'], average='weighted')

print("Combined F1 score:", f1)



In [23]:
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Assuming you have already defined DEVICE and the convert_probabilities and standardise_results functions.

def bart_nli_batched(premises, hypotheses):
    model_name = 'facebook/bart-large-mnli'
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    model.to(DEVICE) 

    # Convert scalar inputs to lists
    if not isinstance(premises, (list, tuple)):
        premises = [premises]
    if not isinstance(hypotheses, (list, tuple)):
        hypotheses = [hypotheses]

    # Tokenize all input pairs together
    inputs = tokenizer(premises, hypotheses, return_tensors='pt', padding=True, truncation=True)
    inputs.to(DEVICE)

    # Run the input through the model
    logits = model(**inputs).logits

    
    return logits


In [41]:
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Assuming you have already defined DEVICE and the convert_probabilities and standardise_results functions.

def bart_nli_batched(premises, hypotheses):
    model_name = 'facebook/bart-large-mnli'
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    model.to(DEVICE) 

    # Convert scalar inputs to lists
    if not isinstance(premises, (list, tuple)):
        premises = [premises]
    if not isinstance(hypotheses, (list, tuple)):
        hypotheses = [hypotheses]

    num_premises = len(premises)
    num_hypotheses = len(hypotheses)
    results = {}

    for i in range(num_premises):
        labels = []
        probabilities_list = []

        print([premises[i]] * num_hypotheses)
        # Tokenize the input pair
        inputs = tokenizer([premises[i]] * num_hypotheses, hypotheses, return_tensors='pt', padding=True, truncation=True)

        # Run the input through the model
        logits = model(**inputs.to(DEVICE)).logits

        # Get probabilities and labels for the input
        probabilities = torch.softmax(logits, dim=1)
        label_mapping = ['contradiction', 'neutral', 'entailment']
        label = label_mapping[probabilities.argmax(dim=1)]

        labels.append(label)
        probabilities_list.append((probabilities))

        results[premises[i]] = (labels, probabilities_list)

    return results


In [43]:
bart_nli_batched(premises, hypotheses)

{'This whole tragedy is the face and body and physical manifestation of fundamental Islam.  Striving to conquer subdue and to bring under control.  Do not blame the world.  Blame fundamental Islam for this tragedy.': ([['entailment',
    'contradiction',
    'contradiction',
    'contradiction',
    'contradiction',
    'contradiction',
    'contradiction',
    'contradiction',
    'contradiction',
    'neutral',
    'contradiction',
    'entailment',
    'entailment',
    'contradiction']],
  [tensor([[0.0231, 0.2139, 0.7630],
           [0.9727, 0.0257, 0.0016],
           [0.9570, 0.0402, 0.0028],
           [0.9869, 0.0107, 0.0023],
           [0.8176, 0.1689, 0.0134],
           [0.4699, 0.4351, 0.0950],
           [0.7749, 0.2211, 0.0039],
           [0.6829, 0.3095, 0.0076],
           [0.8244, 0.1235, 0.0521],
           [0.1786, 0.8131, 0.0083],
           [0.6715, 0.3251, 0.0034],
           [0.0050, 0.0722, 0.9228],
           [0.0342, 0.3565, 0.6094],
           [0.9035, 0.